In [1]:
pip install python-dotenv langchain-openai langchain-community

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

from dotenv import load_dotenv

load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

In [5]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()

In [19]:
response = llm.invoke("Suggest me a title of movie in hindi to watch")
response

AIMessage(content='"दिलचस्प कहानी" (Interesting Story)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 18, 'total_tokens': 39, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-6a45a6fc-6679-43c2-b067-b6e1a541249a-0', usage_metadata={'input_tokens': 18, 'output_tokens': 21, 'total_tokens': 39, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [33]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage("Translate the following from English into Italian"),
    HumanMessage("hi!"),
]
llm.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 4, 'prompt_tokens': 20, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8dad4e47-0bbd-44a0-a18f-16349f61a974-0', usage_metadata={'input_tokens': 20, 'output_tokens': 4, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [31]:
response.content

'Ciao!'

In [35]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following from English into {language}"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [47]:
prompt = prompt_template.invoke({"language": "hindi", "text": "what is your age"})

prompt

ChatPromptValue(messages=[SystemMessage(content='Translate the following from English into hindi', additional_kwargs={}, response_metadata={}), HumanMessage(content='what is your age', additional_kwargs={}, response_metadata={})])

In [49]:
llm.invoke(prompt)

AIMessage(content='तुम्हारी उम्र क्या है?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 22, 'total_tokens': 43, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-93a412b8-2a73-4d8b-8b37-3f2701c30952-0', usage_metadata={'input_tokens': 22, 'output_tokens': 21, 'total_tokens': 43, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [59]:
from langchain_core.prompts import PromptTemplate

prompt_template = PromptTemplate.from_template("Tell me a joke about {topic}")

prompt = prompt_template.invoke({"topic": "cats"})

response = llm.invoke(prompt)

response.content

'Why was the cat sitting on the computer?\n\nBecause it wanted to keep an eye on the mouse!'

In [71]:
movie_title_template = PromptTemplate.from_template("Suggest me title of a movie to watch in language {language} and of genre {genre}")

movie_title_prompt = movie_title_template.invoke({"language": "hindi", "genre":"comedy"})

response = llm.invoke(movie_title_prompt)

response.content

'"Golmaal: Fun Unlimited"'

In [81]:
from langchain_core.output_parsers import StrOutputParser

movie_title_chain = movie_title_template | llm | StrOutputParser()

movie_title_chain.invoke({"language": "english", "genre":"horror"})


'"The Haunting of Hill House"'

In [83]:
movie_summary_prompt = PromptTemplate.from_template("Give me 2-3 line summary of the movie {movie_title}")

In [99]:
from langchain_core.runnables import RunnableLambda

print_title_step = RunnableLambda(lambda x : print(x["movie_title"]))

composed_chain = {"movie_title": movie_title_chain} | print_title_step | movie_summary_prompt | llm | StrOutputParser()

In [103]:
from langchain_core.runnables import RunnableSequence

composed_chain = RunnableSequence({"movie_title": movie_title_chain}, print_title_step, movie_summary_prompt, llm, StrOutputParser())

In [113]:
summary = composed_chain.invoke({"language": "english", "genre":"horror"})
print(summary)

"The Haunting Shadows"
"None" follows a young woman who wakes up in a mysterious facility with no memory of how she got there. As she tries to piece together the events leading up to her arrival, she uncovers shocking truths about her past and the sinister motives of those around her. A tense and psychological thriller that will keep you guessing until the very end.


In [115]:
from langchain_core.runnables import RunnableParallel

translate_hindi_chain = ChatPromptTemplate.from_template("Translate the summary {summary} to hindi") | llm | StrOutputParser()
tranlate_spanish_chain = ChatPromptTemplate.from_template("Translate the summary {summary} to spanish") | llm | StrOutputParser()

translate_runnable = RunnableParallel(hindi_translate = translate_hindi_chain, spanish_translate = tranlate_spanish_chain)

translated_summary = translate_runnable.invoke({"summary" : summary})

print("Hindi Summary: ",  translated_summary["hindi_translate"])
print("Spanish Summary: ", translated_summary["spanish_translate"])



Hindi Summary:  "कोई नहीं" एक युवा महिला का पीछा करती है जो एक रहस्यमय संस्थान में आकर जागती है और उसे यह कैसे पहुँचा यह सम्बंध में कोई याद नहीं है। जब वह अपनी आगमन से पहले के घटनाओं को संजोये की कोशिश करती है, तो उसे अपने भूत पूर्व और उसके आसपास के विश की गंभीर उद्देश्यों के बारे में चौंकानेवाले सत्य पता चलते हैं। यह एक तनावपूर्ण और मानसिक थ्रिलर है जो आखिरी तक आपको गिनती करता रहेगा।
Spanish Summary:  "None" sigue a una joven mujer que despierta en una misteriosa instalación sin memoria de cómo llegó allí. Mientras intenta reconstruir los eventos que la llevaron hasta su llegada, descubre verdades impactantes sobre su pasado y los oscuros motivos de quienes la rodean. Un thriller tenso y psicológico que te mantendrá en vilo hasta el final.
